## Чат-бот на модели FastText

In [1]:
!pip install pymorphy2 stop_words annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.4/647.4 kB 36.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 32.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32897 sha256=04cafef0c9bc99f00ff0f3328755d0b7b3046b30f47b79b1a9f4d7cd2559b384
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
  Created wheel for annoy: filename=annoy-1.17.2-cp310-cp310-linux_x86_64.whl size=582705 sha256=b9dd54e8708d341c5dd597ea0361c7923c9ab766fe019e802d4e5f29659eaf31
  Stored in directory: /root/.cache/pip/wheels/7a/d9/59/473fa56df8e39430eeda369500b4e7127f5b243ba24c3c4297


In [26]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

In [3]:
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/GB_NLP/Chat_Bot/Data/'

### Подготовка данных

In [4]:
import mmap

def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

In [5]:
assert True

#Small preprocess of the answers

question = None
written = False

with open(DATA_PATH + "prepared_answers.txt", "w") as fout:
    with open(DATA_PATH + "Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin, total=get_num_lines(DATA_PATH + "Otvety.txt")):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

<ipython-input-5-c12dbf555266>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin, total=get_num_lines(DATA_PATH + "Otvety.txt")):


  0%|          | 0/7550926 [00:00<?, ?it/s]

In [27]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [7]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0
max_rows = 500000

with open(DATA_PATH + "Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin, total=max_rows):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > max_rows:
            break

<ipython-input-7-d4100d970fa5>:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin, total=max_rows):


  0%|          | 0/500000 [00:00<?, ?it/s]

Сохраним на случай дальнейшего перезапуска

In [8]:
import pickle
with open(DATA_PATH + 'prepared_sents.pkl', 'wb') as f:
   pickle.dump(sentences, f)

In [9]:
with open(DATA_PATH + 'prepared_sents.pkl', 'rb') as f:
  sentences = pickle.load(f)

Возьмем для обучения предложения где больше 2х слов

In [10]:
sentences = [i for i in sentences if len(i) > 2]

FastText

In [11]:
modelFT = FastText(sentences=sentences, vector_size=300, min_count=1, window=5, workers=8)

In [12]:
MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/GB_NLP/Chat_Bot/Models/'

In [13]:
modelFT.save(MODEL_PATH + "ft_model")

In [14]:
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

with open(DATA_PATH + "prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f, total=max_rows):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        vector_ft = np.zeros(300)
        for word in question:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > max_rows:
            break

ft_index.build(10)

<ipython-input-14-518d84432818>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f, total=max_rows):


  0%|          | 0/500000 [00:00<?, ?it/s]

True

In [15]:
ft_index.save(MODEL_PATH + 'speaker.ann')

True

In [16]:
with open(MODEL_PATH + 'index_map.pkl', 'wb') as f:
   pickle.dump(index_map, f)

In [17]:
with open(MODEL_PATH + 'index_map.pkl', 'rb') as f:
  index_map = pickle.load(f)

In [18]:
modelFT = FastText.load(MODEL_PATH + "ft_model")

In [19]:
ft_index = annoy.AnnoyIndex(300, 'angular')
ft_index.load(MODEL_PATH + 'speaker.ann') 

True

In [20]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3, )
    return [index_map[i] for i in answers]

Проверим работу модели

In [21]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [25]:
TEXT = "Сколько звезд на небе?"
get_response(TEXT, ft_index, modelFT, index_map)

['<p> МИЛЛИОН МИЛЛИОН АЛЫХ ЗВЁЗД </p>. \n',
 'Звездочеты наконец сосчитали все звезды <br><br>Австралийские астрономы объявили о том, что в видимой части Вселенной звезд в десять раз больше, чем песчинок во всех пустынях и на всех морских побережьях нашей планеты. <br><br>Ночью с Земли невооруженным взглядом можно различить до 5000 звезд, а с освещенной ночной улицы всего около 100 самых крупных. На самом деле звезд в видимой части Вселенной куда больше, сообщает BBC. <br><br>Австралийские астрономы использовали несколько крупнейших телескопов Земли, чтобы чтобы измерять яркость всех галактик в одном секторе Вселенной и вычислить сколько звезд они содержат. <br><br>Данные этого исследования позволили им назвать приблизительное количество звезд во всей видимой Вселенной. Эта цифра была оглашена на конференции Международного астрономического союза в Сиднее. <br><br>Итак, в видимой части Вселенной находится 70 секстиллионов звезд. Т.е. цифра семь и еще 22 нуля. Ну, короче, вот так: <br>70